In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import matplotlib.pyplot as plt
import numpy as np
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
import os, json, cv2, random
from numpy import load
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode

In [ ]:
classes = ["Common line","Fuel line 1","Fuel line 2","Fuel line 3","Fuel line 4"]
#maxNumOfKeypoints = 16  ## give only if your dataset has keypoints
programName = 'engine_'

In [ ]:
cfg = get_cfg()
cfg.MODEL.MASK_ON = True
#cfg.MODEL.KEYPOINT_ON = True
#cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = True
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4 # Set threshold for this model
#cfg.MODEL.RESNETS.STEM_OUT_CHANNELS = 64
#cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = 64 
#cfg.MODEL.RESNETS.DEPTH = 18
#cfg.MODEL.RESNETS.OUT_FEATURES = ["res2"] 
#cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, False, False, False]
#cfg.MODEL.RESNETS.NORM = "GN"
cfg.MODEL.WEIGHTS = '../Dataset/output/model_final.pth' # Set path model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
#cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = maxNumOfKeypoints
predictor = DefaultPredictor(cfg)

In [ ]:
import cv2
import numpy as np

all_camera_idx_available = []

for camera_idx in range(-2,10):
    print(camera_idx)
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        print(camera_idx)
        print(f'Camera index available: {camera_idx}')
        all_camera_idx_available.append(camera_idx)
        cap.release()

In [ ]:
for d in ["train"]:
    MetadataCatalog.get(programName + d).set(thing_classes=classes)
    #MetadataCatalog.get(programName + d).set(keypoint_names = ["kp1","kp2","kp3","kp4","kp5","kp6","kp7","kp8","kp9","kp10","kp11","kp12","kp13","kp14","kp15","kp16"])
    #MetadataCatalog.get(programName + d).set(keypoint_flip_map = [])
engine_metadata = MetadataCatalog.get(programName+"_train")

In [ ]:
import time
cap = cv2.VideoCapture(4)

if not (cap.isOpened()):
    print("Could not open video device")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1024)
prev_frame_time = time.time()
while(True):    
    # Capture frame-by-frame    
    
    ret, frame = cap.read()
    outputs = predictor(frame)
    
    v = Visualizer(frame[:, :, ::-1],
                   metadata=engine_metadata, 
                   scale=0.9,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("prediction",v.get_image()[:, :, ::-1])
    
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        break
    
    new_frame_time = time.time()
    seconds = new_frame_time - prev_frame_time
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    print("fps",fps)
    #Waits for a user input to quit the application    

    
cap.release()
cv2.destroyAllWindows()